In [101]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import lower, col



In [102]:
#Connection path to the java database connection jars for MongoDB
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars "C:\\MongoDBSparkCOnnector\\mongo-spark-connector_2.12-3.0.2.jar,C:\\MongoDBSparkConnector\\mongo-java-driver-3.12.9.jar" pyspark-shell'

In [103]:
#set up spark session
spark = SparkSession.builder \
    .appName("MongoDB and PySpark") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/GoT.twitter") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/GoT.twitter") \
    .getOrCreate()

In [104]:
#Read in montly data csv files

jan = spark.read.csv('C:\\data files\\Jan2016 GOT.csv',inferSchema=True, header =True)
feb = spark.read.csv('C:\\data files\\Feb2016 GOT.csv')
march = spark.read.csv('C:\\data files\\March2016 GOT.csv')
april = spark.read.csv('C:\\data files\\April2016 GOT.csv')
may = spark.read.csv('C:\\data files\\May2016 GOT.csv')
june = spark.read.csv('C:\\data files\\June2016 GOT.csv')
july = spark.read.csv('C:\\data files\\July2016 GOT.csv')
august = spark.read.csv('C:\\data files\\August2016 GOT.csv')
september = spark.read.csv('C:\\data files\\Sept2016 GOT.csv')
october = spark.read.csv('C:\\data files\\Oct2016 GOT.csv')
november = spark.read.csv('C:\\data files\\Nov2016 GOT.csv')
december = spark.read.csv('C:\\data files\\Dec2016 GOT.csv')

In [105]:
jan.show()

+--------------------+-----------------+------------+--------------------+------------------+--------------------+---------------+--------------+
|              poster|        recipient|relationship|          tweet date|          tweet id|               tweet|       hashtags|retweet status|
+--------------------+-----------------+------------+--------------------+------------------+--------------------+---------------+--------------+
|        alekss______|          nelbka5|    mentions|fri jan 01 07:31:...|682826571724537856|rt @nelbka5: Гото...|['ночьшерлока']|          True|
|      lesmaecarino26|    thrones_memes|    mentions|fri jan 01 07:34:...|682827255400148992|rt @thrones_memes...|             []|          True|
|             1di_kit|        otradaily|    mentions|fri jan 01 07:36:...|682827800659771392|"rt @otradaily: "...|           null|          null|
|""louis and danie...|             null|        null|                null|              null|                null|          

In [106]:
#append into one dataframe
got_df = jan.union(feb).union(march).union(april).union(may).union(june).union(july).union(august).union(september).union(october).union(november).union(december)

In [107]:
got_df.count()

352773

In [108]:
got_df.printSchema()

root
 |-- poster: string (nullable = true)
 |-- recipient: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- tweet date: string (nullable = true)
 |-- tweet id: string (nullable = true)
 |-- tweet: string (nullable = true)
 |-- hashtags: string (nullable = true)
 |-- retweet status: string (nullable = true)



In [109]:
#Get rid of false positives
#Drop all where hashtag != gameofthrones

got_df = got_df.withColumn("hashtags", lower(col("hashtags")))
got_df = got_df.filter(got_df.hashtags.contains("gameofthrones"))


In [110]:
got_df.show()

+---------------+---------------+------------+--------------------+------------------+--------------------+--------------------+--------------+
|         poster|      recipient|relationship|          tweet date|          tweet id|               tweet|            hashtags|retweet status|
+---------------+---------------+------------+--------------------+------------------+--------------------+--------------------+--------------+
|    irinaforden|       j0n_snow|    mentions|fri jan 01 09:36:...|682857802516172800|rt @j0n_snow: you...|['jonsnow', 'game...|          True|
|    fantasysite|    fantasysite|       tweet|fri jan 01 10:57:...|682878425925746688|#gameofthrones ga...|   ['gameofthrones']|         False|
|alandalustravel|          spain|    mentions|fri jan 01 14:25:...|682930607253499904|"rt @spain ""#did...|['didyouknow', 'l...|         False|
|     rockxx1974|     rockxx1974|       tweet|fri jan 01 14:36:...|682933518113046528|#gameofthrones ma...|['gameofthrones',...|        

In [111]:
#Store in MongoDB
got_df.write.format("mongo").mode("append").save()



In [112]:
 spark.stop()